<a href="https://colab.research.google.com/github/whykusanagi/vThree_Sizes/blob/main/Vtuber_3_Sizes_Estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# vThree_Sizes Character Measurement Calculation
Calculator for vtubers to determine their 3 sizes accurately.
This notebook processes an anime character image to extract height, bust, waist, underbust, and hip measurements using Mediapipe Pose.

Launch in google collab and upload your images directly in `/content/` for easy analysis.

### Conversion Explanation
- The reference character height is set to **175 cm**. As my character is tall and based on the male vtuber model she is derived from.
- The **Nude** height in pixels (1410 pixels) is used as the baseline for conversion.
- The **cm/pixel ratio** is calculated as:
  
  \[
  cm\_per\_pixel = \frac{175}{1410} \approx 0.1241 \text{ cm/pixel}
  \]

You can change the calculation as needed, but if your character is smaller you can adjust the parameter to the average height.
The average American woman is approximately 5 feet 4 inches (162.5 cm) according to the CDC.

### Cup Size Calculation
- Cup size is determined by the difference between the bust and underbust measurements, following standard thresholds:
  - **AA**: 1.0-2.5 cm
  - **A**: 2.6-3.5 cm
  - **B**: 3.6-5.0 cm
  - **C**: 5.1-6.5 cm
  - **D**: 6.6-8.0 cm
  - **DD/E**: 8.1-9.5 cm
  - **F**: 9.6-11.0 cm
  - **G**: 11.1-12.5 cm
  - **H**: 12.6-14.0 cm

### Output Data Table
The final table provides measurements in centimeters for each outfit variation, including the estimated cup size.


If you found this tool helpful, donate so I can spend more time building random projects like this for the community.
![Donate](https://s3.whykusanagi.xyz/Simple_Header_Image.png)(https://streamelements.com/whykusanagi/tip)

In [ ]:
!pip install mediapipe

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import pandas as pd
from measurements import (
    MeasurementConfig,
    centimeters_per_pixel,
    estimate_body_circumferences,
    estimate_cup_size,
)

In [ ]:
import os
from mediapipe.python.solutions.pose import PoseLandmark

# Path to images folder
image_folder = "/content/"

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=2)
config = MeasurementConfig(reference_height_cm=175.0)


def process_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    return results, image


def extract_keypoints(results, image_shape):
    if not results or not results.pose_landmarks:
        return None
    h, w, _ = image_shape
    return {i: (int(lm.x * w), int(lm.y * h)) for i, lm in enumerate(results.pose_landmarks.landmark)}


def compute_raw_height(keypoints):
    nose = keypoints[PoseLandmark.NOSE.value]
    ankle = keypoints[PoseLandmark.LEFT_ANKLE.value]
    return abs(nose[1] - ankle[1])


def process_folder(folder):
    files = sorted(f for f in os.listdir(folder) if f.lower().endswith(("png", "jpg", "jpeg")))
    measurements = {}
    for image_file in files:
        image_path = os.path.join(folder, image_file)
        results, image = process_image(image_path)
        keypoints = extract_keypoints(results, image.shape)
        if not keypoints:
            print(f"No pose detected in {image_file}")
            continue

        height_px = compute_raw_height(keypoints)
        try:
            cm_per_pixel = centimeters_per_pixel(height_px, config.reference_height_cm)
        except ValueError as exc:
            print(f"Skipping {image_file}: {exc}")
            continue

        circs = estimate_body_circumferences(keypoints, cm_per_pixel, config)
        converted = {
            "Height (cm)": round(height_px * cm_per_pixel, 1),
            **circs,
        }
        converted["Cup Size"] = estimate_cup_size(
            converted["Bust Circumference"],
            converted["Underbust Circumference"],
        )
        measurements[image_file] = converted
    return measurements


all_measurements = process_folder(image_folder)
df = pd.DataFrame.from_dict(all_measurements, orient='index')
print(df)

                  Height  Bust Circumference  Waist Circumference  \
no_dress.png       137.4                49.5                 32.9   
marie_dress.png    175.0                45.8                 32.8   
nude_body.png      175.0                48.2                 32.9   
plug_suit.png      177.9                47.0                 34.5   
bunny_outfit.png   179.8                50.9                 35.1   

                  Underbust Circumference  Hip Circumference Cup Size  
no_dress.png                         32.0               32.9        J  
marie_dress.png                      31.8               32.8        H  
nude_body.png                        32.0               32.9        J  
plug_suit.png                        33.5               34.5        H  
bunny_outfit.png                     34.1               35.1        J  
